## Load up the environment variable needed for module imports

In [28]:
%env MIRA_REST_URL=http://34.230.33.149:8771

env: MIRA_REST_URL=http://34.230.33.149:8771


## Import modules

In [29]:
from mira.sources import biomodels
from mira.metamodel.comparison import *
from mira.metamodel.template_model import *
from mira.metamodel.templates import *
from mira.sources import biomodels

from itertools import combinations
from copy import deepcopy

## Define the Covid epi models to be compared and add them to a list 

In [ ]:
COVID_MODEL_SET = {"BIOMD0000000955", "BIOMD0000000956", "BIOMD0000000957","BIOMD0000000958","BIOMD0000000960","BIOMD0000000962"}
rc = get_dkg_refinement_closure()

tm_covid_grounding_list = []
for covid_model in COVID_MODEL_SET:
    tm_covid_grounding_list.append(biomodels.get_template_model(covid_model))

## Compute pairwise similarity scores between each grounded model

In [ ]:
tm_covid_comparison = TemplateModelComparison(tm_covid_grounding_list,refinement_func=rc.is_ontological_child)
tm_covid_comparison.model_comparison.get_similarity_scores()

## Create image comparing selected grounded models

In [ ]:
TemplateModelDelta.for_jupyter(tm_covid_comparison.template_models[2],tm_covid_comparison.template_models[5],
                                               rc.is_ontological_child,name=f"model_{2}{5}_grounding.png",
                                                  args="-Grankdir=TB")

## Create a list of covid models without grounding 

In [ ]:
tm_covid_no_grounding_list = [] 
for tm in tm_covid_grounding_list:
    copied_tm = deepcopy(tm)
    for template in copied_tm.templates:
        for concept in template.get_concepts():
            concept.identifiers = {}
            concept.context = {}
    tm_covid_no_grounding_list.append(copied_tm)

## Compute pairwise similarity scores between each ungrounded model

In [ ]:
tm_covid_comparison_copy = TemplateModelComparison(tm_covid_no_grounding_list,refinement_func=rc.is_ontological_child)
tm_covid_comparison_copy.model_comparison.get_similarity_scores()

## Create image comparing selected ungrounded models

In [ ]:
TemplateModelDelta.for_jupyter(tm_covid_comparison_copy.template_models[2],tm_covid_comparison_copy.template_models[5],
                                               rc.is_ontological_child,name=f"model_{2}{5}_no_grounding.png",
                                args="-Grankdir=TB")

## Reasoning on difference between grounded and ungrounded models

The similarity score for models 2 and 5 when grounded is 0.75. The similarity score for the models when ungrounded is 0.25. The reasoning for this is because the refinements dissapear and the confirmed concept node is no longer the same between the two models.